** Amazon Product Analysis, including Review Analysis
The .csv has been imported in the first project file to create a star schema image, attached in the folder. 

In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

engine = create_engine('sqlite:///amazon_sales.csv')

csv_filepath = 'data/amazon_sales.csv'
raw_df = pd.read_csv(csv_filepath)

raw_df.to_sql('stg_amazon_products_reviews_raw', engine, if_exists='replace', index=False)

print("Raw data loaded successfully into the 'stg_amazon_products_reviews_raw' table.")

Raw data loaded successfully into the 'stg_amazon_products_reviews_raw' table.


In [2]:
%load_ext sql
%sql sqlite:///amazon_sales.csv

In [ ]:
%%sql

CREATE TABLE dim_product(
    product_key INT PRIMARY KEY,
    product_id VARCHAR(20) NOT NULL UNIQUE, 
    product_name VARCHAR(50) NOT NULL,
    category TEXT,
    discounted NUMERIC,
    actual_price NUMERIC,
    discount_percentage NUMERIC, 
    product_link LINK
);

In [14]:
%%sql

ALTER TABLE dim_product
DROP COLUMN product_key;

 * sqlite:///amazon_sales.csv
(sqlite3.OperationalError) cannot drop PRIMARY KEY column: "product_key"
[SQL: ALTER TABLE dim_product
DROP COLUMN product_key;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [4]:
%%sql

INSERT INTO dim_product(
    product_id, product_name, category, actual_price, discount_percentage, product_link
    )
SELECT
    t1.product_id,
    t1.product_name,
    t1.category,
    t1.actual_price,
    t1.discount_percentage,
    t1.product_link
FROM (
    SELECT
        product_id,
        product_name,
        category, 
        actual_price,
        discount_percentage,
        product_link
    FROM stg_amazon_products_reviews_raw
    GROUP BY 1, 2
) AS t1;

 * sqlite:///amazon_sales.csv
(sqlite3.IntegrityError) NOT NULL constraint failed: dim_product.product_key
[SQL: INSERT INTO dim_product(
    product_id, product_name, category, actual_price, discount_percentage, product_link
    )
SELECT
    t1.product_id,
    t1.product_name,
    t1.category,
    t1.actual_price,
    t1.discount_percentage,
    t1.product_link
FROM (
    SELECT
        product_id,
        product_name,
        category, 
        actual_price,
        discount_percentage,
        product_link
    FROM stg_amazon_products_reviews_raw
    GROUP BY 1, 2
) AS t1;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


Fact table for review metrics is to be created last, then will be populated using the staging table. 

In [ ]:
%%sql

CREATE TABLE fact_review_metrics(
    review_key INT PRIMARY KEY NOT NULL,
    date_key DATE NOT NULL,
    user_key VARCHAR(30) NOT NULL,
    product_key VARCHAR(30) NOT NULL,
    rating METRIC,
    rating_count METRIC,
    rating_title DESCRIPTIVE
);

In [ ]:
%%sql

INSERT INTO fact_review_metrics(
    user_key, review_key, date_key, product_key, rating, rating_count, rating_title
)
WITH transformed_date AS (
    SELECT
        R.rating,
        R.rating_count,
        DU.user_key,
        DP.product_key,
    FROM stg_amazon_products_reviews_raw AS R
    INNER JOIN
)

 * sqlite:///amazon_sales.csv
(sqlite3.OperationalError) incomplete input
[SQL: INSERT INTO fact_review_metrics(
    user_key, review_key, date_key, product_key, rating, rating_count, rating_title
)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
